# DataFrame Missing Value Methods

This chapter covers the methods available to a DataFrame to handle missing values. They are the exact same methods that were presented for the Series in the last part. These methods behave analogously as they do with Series, but are more complex as they deal with missing values in all of the columns in the DataFrame and have more options for their usage. We begin b reading in the movie dataset setting the title as the index.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 40)
movie = pd.read_csv('../data/movie.csv', index_col='title')
movie.head(3)

## Methods for handling missing values

pandas provides the following DataFrame methods to handle missing values:

* `isna` - Returns a DataFrame of booleans based on whether each value is missing or not
* `notna` - Exact opposite of `isna`
* `fillna` - Fills missing values in a variety of ways
* `dropna` - Drops the missing values from the Series
* `interpolate` - Fills missing values with statistical interpolation

## The `isna` method

The `isna` method returns a DataFrame of all boolean values. All missing values evaluate as `True` with everything else evaluating as `False`. Let's call this method on our `movie` DataFrame.

In [ ]:
movie.isna().head(3)

### Finding the number of missing values in each column

There is no single direct method for finding the number of missing values in each column of a DataFrame. However, the `count` method returns the number of non-missing values of each column. Let's see an example of that here.

In [ ]:
movie.count().head()

To find the number of missing values we will need to first call the `isna` method to turn each value into a boolean and then chain the `sum` method to count the number of missing values in each column.

In [ ]:
movie.isna().sum().head()

### Find the percentage of missing values by calling the mean method

As we have seen before, taking the mean of a boolean Series returns the percentage of values that are `True` and in this case returns the percentage of missing values for each column.

In [ ]:
movie.isna().mean().head()

### Nicer visuals with rounding

Although the above result is accurate, it isn't taht easy to read. The last few decimal places provide little information and are not necessary to report. The result isn't technically a percentage either, but a fraction. Let's round the values and multiply by 100 to get a nicer visual report that is a percentage.

In [ ]:
movie.isna().mean().round(3).head() * 100

### The `notna` method

The `notna` method is the exact opposite of `isna` and evaluates as `True` for all non-missing values.

In [ ]:
movie.notna().head(3)

## Dropping rows and columns with the `dropna` method

The `dropna` method has more options that its Series counterpart. It allows you to drop either rows or columns of your DataFrame in a variety of ways. Called with the defaults, it drops all **rows** that have at least 1 missing value. Over 1,000 rows get dropped after calling the `dropna` method on our `movie` DataFrame.

In [ ]:
movie.dropna().shape

### Drop rows where only a particular column is missing

By default, the `dropna` method drops any rows where there are one or missing values for that entire row. pandas gives us the option of only dropping rows where a particular column or columns have missing values. Instead of looking at all of the columns for missing values, we can restrict the columns with the `subset` parameter. Set this parameter to a list of the column names to inspect for missing values. Below, we only drop rows that are missing either the `year` or `content_rating`.

In [ ]:
movie.dropna(subset=['year', 'content_rating']).shape

In [ ]:
movie.shape

Another option for `dropna` is setting the minimum threshold for non-missing values in each row. Our current DataFrame has 21 columns. Below, we use the `thresh` parameter to ensure that each row has at least 18 non-missing values.

In [ ]:
movie.dropna(thresh=18).shape

### Drop columns with missing values

Just like we did with the statistical methods, we can change the direction of the operation for `dropna` and drop columns instead of rows by setting the `axis` parameter to 1. Let's drop all columns that have 1 or more missing values.

In [ ]:
movie.dropna(axis=1).head()

The `subset` parameter is available when dropping columns and can be given a list of index labels to limit the rows to look for missing values. The `thresh` parameter can be given an integer to reprsent the minimum number of non-missing values in a particular in order to be kept in the result. Here we drop all of the columns that don't have at least 4,800 non-missing values.

In [ ]:
movie.dropna(axis=1, thresh=4800).shape

## Filling missing values with the `fillna` method

The `fillna` method fills the missing values in your DataFrame in a couple of different ways.

### Filling the missing values with a given constant

The most basic way to use the `fillna` method is to pass it a single value which replaces every missing value with this constant. The following replaces all missing values with the string 'FILLED'. Note, that this replaces all missing value representations - `NaN`, `NaT`, and `None`. We only output the fifth row as it was the only one of the first five to contain missing values.

In [ ]:
movie.fillna('FILLED').iloc[4:5, :]

Filling a DataFrame's missing values with a single constant value is unlikely what you'd need in a real situation. The `movie` DataFrame has columns of different data types and by using the string 'FILLED' we will have changed the data type of any numeric column to object. For instance, the column `duration` was originally a float and is now an object.

### Use a dictionary to fill specific columns

A more practical application would be to fill each column with a different constant value. We can use `fillna` to do this by passing it a dictionary that maps the column name to the missing value replacement. The following fills the `content_rating` column with 'PG' and the `duration` column with 199. Missing values in other columns are not filled.

In [ ]:
movie.fillna({'content_rating': 'PG', 'duration': 199}).iloc[4:5]

### Fill all columns with the mean or median

A somewhat common approach to filling missing values is to use the mean or median of the column as the replacement. Taking the `median` of a DataFrame returns a Series that has each column name labeling its median. A Series is also a valid object that can be passed to the `fillna` method. Let's begin by finding the median of each column.

In [ ]:
mm = movie.median(numeric_only=True)
mm

Pass the above Series to `fillna` to use a different missing value for each column. The string columns will not be filled as they do not have a median.

In [ ]:
movie.fillna(mm).iloc[4:5]

Filling missing values with the mean isn't necessarily a good strategy when doing data analysis. The example above is merely used to demonstrate how the `fillna` method works.

### Filling missing values with the preceding or following values
Instead of filling missing values with a constant, you can fill in missing values with the immediately preceding or following known value. Let's use a simple csv to clearly see how this done.

In [ ]:
df = pd.read_csv('../data/missing_example.csv')
df

The `method` parameter controls whether you are going to fill in the missing value with the immediate preceding or following value. Set it equal to the string 'ffill' to fill missing values going 'forward', i.e. using the immediate preceding value.

In [ ]:
df.fillna(method='ffill')

Notice that the first value for the `orders` column is still missing as there is no immediate preceding value to fill it with. Use the string 'bfill' to fill missing values going backwards.

In [ ]:
df.fillna(method='bfill')

You can limit the number of consecutive values filled with the `limit` parameter. Here we limit the number filled to only 1.

In [ ]:
df.fillna(method='ffill', limit=1)

When you use `fillna`, you must choose between filling with a constant demonstrated by the first example or by forward or backfilling with the `method` parameter, shown in the second example.

## The `interpolate` method

The `interpolate` method was covered extensively in the Series part. It works the same with DataFrames and fills in missing values using a variety of statistical methods. By default, it uses linear interpolation on every column in the DataFrame. Let's read a small sample of stock price data for Microsoft, Apple, and Schlumberger with several missing values.

In [ ]:
df_stocks = pd.read_csv('../data/stocks/sample_missing.csv')
df_stocks

Let's use linear interpolation to fill in the missing values of each column.

In [ ]:
df_stocks.interpolate('linear')

Here, we use quadratic interpolation and limit 

In [ ]:
df_stocks.interpolate('quadratic', limit=1)

## Exercises

Execute the next cell to read in the college dataset with the institution name as the index and use it for the first few exercises.

In [ ]:
college = pd.read_csv('../data/college.csv', index_col='instnm')
college.head(3)

The data dictionary is helpful to decipher the meaning of the columns. Uncomment the next line to read it in.

In [ ]:
# pd.read_csv('../data/college_data_dictionary.csv')

### Exercise 1

<span  style="color:green; font-size:16px">Find the number of missing values for each row.</span>

### Exercise 2

<span  style="color:green; font-size:16px">What percentage of rows have more than 5 missing values?</span>

### Exercise 3

<span  style="color:green; font-size:16px">How many total missing values are there in the entire DataFrame?</span>

### Exercise 4

<span  style="color:green; font-size:16px">How many total non-missing values are there in the entire DataFrame?</span>

### Exercise 5

<span  style="color:green; font-size:16px">How many rows will be dropped when the `dropna` method is called with its defaults. Calculate this number without calling the `dropna` method.</span>

### Exercise 6

<span  style="color:green; font-size:16px">Verify the result from exercise 5 by calling the `dropna` method.</span>

### Exercise 7

<span  style="color:green; font-size:16px">Drop all the rows that are missing the `ugds` column.</span>

### Exercise 8

<span style="color:green; font-size:16px">Drop all columns that have more than 5% of their values missing.</span>

### Exercise 9

<span style="color:green; font-size:16px">Fill in the missing values with the maximum value of each column.</span>